The Data sources are:

### 1. Liam's SPSS coded data
**File:** The Loop 2017 Final Interventions.xlsx

Exported as Excel from SPSS, keeping the variable names.

This file contains 1325 entries.


27 have null festival or sample numbers so can't be used, leaving 1298


One has sample number 12151, two have sample number 0 - these cannot be merged.


This leaves 1295 - all of which can be merged

### 2. Guy's cleaned up lab data
**File:** Loop 2017 Lab fixed data.xlsm

From: Dropbox/Testing/2017 results processing/Loop 2017 Lab fixed data.xlsm


Data is in the ‘Raw Lab Data’ sheet


This file contains 2544 entries


1900 entries start with F


621 entries begin with A (amnesty) so can't be merged


23 Begin with W? so can't be merged


Entry SGP2017 F0465 needs editing as 'Client gender' is FemaleaMalee 

### 3. Boomtown Intervention Questionnaire
**File:** BTReport 2017 - Form responses 3.csv

Exported from: https://docs.google.com/spreadsheets/d/15pdETY0HK-VbBcV-N0swt6ZrRBbeDnZR5RGDzfq95dg

This file contains 194 entries

### 4. 'Straggling' Boomtown Intervention Questionnaire
**File:** Reports V2.6 Branch 2 - Form responses 2.csv

https://docs.google.com/spreadsheets/d/1sZXFdiOaUX6n9HGq9s-t8T_zxNZhKyjxY83aY8mvUIo/edit#gid=1291806732

This file contains 9 entries

### Merging the data

Merging the data on Festival and SampleNumber resulted in 1295 entries



In [1]:
# Module imports
import os
import numpy as np
import pandas as pd

def fix_sample_number(x):
    if isinstance(x, float) and np.isnan(x):
        return x # leave NaN's alone    
    try:
        sn = int(x)
        sn = 'F{:04d}'.format(int(x))
    except ValueError:
        # Assume string so make sure it's of the right format
        sn = str(x).capitalize()
    return sn

In [2]:
spssdata = '/opt/random/The Loop 2017 Final Interventions.xlsx'

spss_df = pd.read_excel(spssdata)
print("Read in from SPSS ",len(spss_df))

# Change festival names
spss_df['Festival'].replace(['BoomTown', 'KC', 'SGP'], ['BT2017', 'KC2017', 'SGP2017'], inplace=True)

# Fix broken column labels

d = {'spice_yesterday' : 'spice_legals_yesterday',
     'Cannabis_ever' : 'cannabis_ever',
     'Ethnictiy_other' : 'Ethnicity_other'
    }
spss_df.rename(columns=d, inplace=True)

# Ensure all Sample numbers are consistent
# 1. Delete any rows where SampleNumber or Festival is NA as we can't do anything with it
# There are 45 entries that go, but none of them have any valid data in
spss_df.dropna(subset=['SampleNumber', 'Festival'], inplace=True)

# 2. Make all sample numbers a 4-digit code starting with F
spss_df['SampleNumber'] = spss_df['SampleNumber'].apply(fix_sample_number)

# Combine date and time columns into new single column
spss_df['Date'] = pd.to_datetime(spss_df['Date']) # Convert Date to datetime object
spss_df['Date & Time of intervention'] = spss_df.apply(lambda r : pd.datetime.combine(r['Date'], r['Time']), 1)

# Remove Day, Date, Time and SurveyID columns
spss_df.drop(['Day', 'Date', 'Time', 'SurveyID'], axis=1, inplace=True)

# Fix dodgy sample number - Guy confirmed this with Liam
spss_df.at[spss_df['SampleNumber'] == 'F12151', 'SampleNumber'] = 'F1215'

print("Final SPSS ",len(spss_df)) # shows we are left with 1298 datasets

Read in from SPSS  1343
Final SPSS  1298


In [3]:
# Code to read the straggling results
bt_straggling = 'Reports V2.6 Branch 2 - Form responses 2.csv'
date_cols = ['Timestamp']
bt_df2 = pd.read_csv(bt_straggling, engine="python", parse_dates=date_cols)

# 2 columns are missing so need to be added as Nan
bt_df2['Which drugs have you used? [Non-prescribed opiods]'] = np.nan
bt_df2['Are you planning to take any of these drugs later?'] = np.nan

# Sample 'F0001' and 'F0535' appear to be errors - one taken by Mike Capper - another a 13 year old female
bt_df2.drop(bt_df2[(bt_df2['Sample Number'] == '1') | (bt_df2['Sample Number'] == 'F0535')].index, inplace=True)

# Delete the columns that are only in straggling results
to_drop = ['Which risks are you aware of that exist when using this substance?', 
           'Please could you tell me exactly what you have had to drink today: [Wine]',
           'The service user abandoned the intervention before completion.',
           'Please could you tell me exactly what you have had to drink today: [Beer]',
           'Please could you tell me exactly what you have had to drink today: [Spirits]',
           'Please could you tell me exactly what you have had to drink today: [Alcopops]',
           'Do you know any ways to reduce those risks?',
           'Did you understand the disclaimer explaining the limitations that was read to you?',
           'Which of the drugs are you planning to use later today?']
bt_df2.drop(to_drop, axis=1, inplace=True)

In [6]:
# Read in and process the columns  
bt_interventions = '/opt/random/BTReport 2017 - Form responses 3.csv'
date_cols = ['Timestamp']
bt_df = pd.read_csv(bt_interventions, engine="python", parse_dates=date_cols)

# Add the 'straggling' results
bt_df = pd.concat([bt_df, bt_df2], axis='rows', ignore_index=True)

# Map Columns in Google Forms to SPSS
d = {'Sample Number': 'SampleNumber',
 'Timestamp' : 'Date & Time of intervention',
 'Number of friends present with primary respondent': 'FriendsPresent',
 'Gender of primary respondent': 'Gender',
 'Ethnicity': 'Ethnicity',
 'Age': 'Age',
 'Have you had any alcohol to drink today?': 'ConsumedAlcohol',
 'How much spirits have you had today?': 'Spirits',
 'How much wine have you had today?': 'Wine',
 'How many alcopops have you had today?': 'Alcopops',
 'Are you currently taking any prescribed medication?': 'PrescribedDrugs',
 'Are you currently taking any "Over the Counter" medication?': 'OverTheCounter',
 'Do you have any concerns about how you are feeling at the moment?': 'ConcernsWithCurrentFeelings',
 'You submitted a substance of concern for analysis, what do you believe it to be?': 'SubmittedSubstanceAs',
 'Where did you obtain the sample?': 'Obtained',
 'Very roughly, how often do you use this drug?': 'EverHadSubstance',
 'When did you first use this batch?': 'WhereAndWhen',
 'Have you or anyone you know ever had negative experiences taking this substance?': 'NegativeExperieces',
 'How many times have you used this batch?': 'ConsumedFromBatchAlready',
 'Do you have any concerns about using this sample from this batch or any other concerns about the result?': 'PriorConcerns',
 'Have you ever accessed a treatment service for your alcohol or drug use?': 'AccessedSupportBefore',
 'After our conversation today, would you like to have any further advice or support from a treatment service for your alcohol or drug use?': 'WantFurtherAdvice',
 # Need to check this one
 'Have you ever taken any other drugs I didn\'t mention?' : 'other_specify'
}

bt_df.rename(columns=d, inplace=True)

# Delete all columns that are only in Google Forms
to_drop = ['Volunteer Name', 
    'When was the last time you used this service?',
    'What was your first sample number at this event? Did you take a photo or keep the ticket?',
    'Which drugs have you used? [Non-prescribed opiods]',
    'Have you had any other legal or illegal drugs today?',
#    'Have you ever taken any other drugs I didn\'t mention?',
    'Are you planning to take any of these drugs later?']

bt_df.drop(to_drop, axis=1, inplace=True)

# Add Festival Column
bt_df['Festival'] = 'BT2017'

# Add As_expected Column - is all null in Liams
bt_df['As_expected'] = np.nan

# Fix SampleNumber
bt_df['SampleNumber'] = bt_df['SampleNumber'].apply(fix_sample_number)

# Fix the broken sample numbers - have already made sure they're not in the bt_interventions set 
#SNBAD = 'F00119'
# SNNEW = 'F0119'
# print("GF ", bt_df.loc[bt_df['SampleNumber'] == SNBAD, ['Gender', 'Age', 'SubmittedSubstanceAs', 'Date & Time of intervention']])
# print("LAB ", lab_df.loc[(lab_df['SampleNumber'] == SNNEW) & (lab_df['Festival'] == 'BT2017'), ['Client gender', 'Client age', 'Bought as', 'Date & Time of return']]) 
# print("SPSS ", spss_df.loc[(spss_df['SampleNumber'] == SNNEW) & (spss_df['Festival'] == 'BT2017'), ['Gender', 'Date & Time of intervention', 'Age', 'SubmittedSubstanceAs']])

# '000-04' - assume 'F0004' as in lab data and date/time of return/intervention are ~ 30 min
bt_df.at[bt_df['SampleNumber'] == '000-04', 'SampleNumber'] = 'F0004'

# '5f009' - 'F0059' as nothing in SPSS and date/time of return/intervention are ~ 5 min
bt_df.at[bt_df['SampleNumber'] == '5f009', 'SampleNumber'] = 'F0059'

# 'F00117' - 'F0117' as nothing in SPSS and date/time of return/intervention are ~ 30 min
bt_df.at[bt_df['SampleNumber'] == 'F00117', 'SampleNumber'] = 'F0117'

# 'F00119' - 'F0119' as nothing in SPSS and date/time of return/intervention are ~ 30 min and samples math
bt_df.at[bt_df['SampleNumber'] == 'F00119', 'SampleNumber'] = 'F0119'

# Sample 'F308470234987' is rubbish
bt_df.drop(bt_df[bt_df['SampleNumber'] == 'F308470234987'].index, inplace=True)

print("DONE COLUMNS")

DONE COLUMNS


In [7]:
#
# Attempt to disentangle the Form responses into a form they can be merged with Liam's data
#
PERIODS = ['ever', 'year', 'month', 'week', 'yesterday', 'today', 'tonight', 'today_tonight']

def includes_frequency(cell, period):
    "Return boolean indicating if this cell contains frequencies >= period"
    if isinstance(cell, float) and np.isnan(cell):
        return False # nan's is considered not having the value
    
    #periods = ['ever', 'year', 'month', 'week', 'yesterday', 'today', 'tonight']
    form_responses = ['Had in my life', 'Had in last year', 'Had in last month', 'Had in last week', 
                      'Had yesterday', 'Had today', '(Probably) planning later']
    
    assert period in PERIODS, "Invalid period: {0}".format(period)
    
    values = [v.strip() for v in cell.split(',')]   
    idx = PERIODS.index(period)
    # check if any of the periods >= this have been checked
    for i in range(idx, len(PERIODS) - 1):
        if form_responses[i] in values:
            return True
    return False

def get_value(column, period):
    "Return boolean Series indicating if this columns contains frequencies >= period"
    result = None
    if period == 'today_tonight':
        today = column.apply(includes_frequency, period='today')
        tonight = column.apply(includes_frequency, period='tonight')
        result = today | tonight
    else:
        result = column.apply(includes_frequency, period=period)
    return result

def add_columns(bt_df, label_map):
    """For all of the drugs in the label_map expand them out to match Liam's data and then delete the original column
    """
    for gdrug in label_map.keys():
        gcolumn_label = 'Which drugs have you used? [{}]'.format(gdrug)
        gcolumn = bt_df[gcolumn_label]
        sdrug = label_map[gdrug]
        # Add columns for each period
        for period in PERIODS:
            column_name = '{}_{}'.format(sdrug, period)
            bt_df[column_name] = get_value(gcolumn, period)

        # Delete the original column
        bt_df.drop([gcolumn_label], axis=1, inplace=True)

def freq_summary(drugs, column_prefix):
    """Run boolean OR (any) for the give set of drugs and put result in column with 
    name {period}_{column_prefix}"""
    for period in PERIODS:
        labels = ['{}_{}'.format(drug, period) for drug in drugs]
        clabel = '{}_{}'.format(column_prefix, period)
        bt_df[clabel] = bt_df[labels].any(axis=1)
     
column_map_core = {'Cannabis' : 'cannabis',
                   'Cocaine' : 'cocaine',
                   'Ecstasy pills' : 'ecstasy',
                   'Nitrous (NOS, laughing gas)' : 'nitrous_oxide',
                   'MDMA crystal/powder' : 'mdma',
                   'Ketamine' : 'ketamine',
                   'Magic mushrooms' : 'mushrooms',
                   'LSD' : 'lsd',
                   'Mephedrone (M-Cat)' : 'Mephedrone',
                   'Synthetic cannabinoids ("Spice")' : 'spice_legals',
                   'A powder which I had no idea what it was' : 'unknown_powder',
                  }

# Add core drug columns
add_columns(bt_df, column_map_core)

# Extra columns that aren't present in Liam's data
column_map_extra = {'2C-B' : '2cb',
                    'Amphetamine (speed)' : 'speed',
                    'Codeine' : 'coedine',
                    'Valium or other benzodiazepines' : 'valium',
                   }
add_columns(bt_df, column_map_extra)


# Now need to add calculated data for the other drug uses:
# Legal (any_legal):    balloons, poppers, spice, other legal highs
# core drugs (core):    cannabis, cocaine, ecstasy, mdma, ketamine, mephodrone, speed, heroin
# polydrug ():    2 + illegal drugs
# polysubstance:    2 + illegal drigs and usual alcohol frequency      

all_drugs = list(column_map_core.values()) + list(column_map_extra.values())
# ['cannabis', 'cocaine', 'ecstasy', 'nitrous_oxide', 'mdma', 'ketamine', 'mushrooms', 'lsd', 'Mephedrone', 'spice_legals', 'unknown_powder', '2cb', 'speed', 'coedine', 'valium']
freq_summary(all_drugs, 'any')
        
legal_drugs = ['nitrous_oxide', 'spice_legals', 'coedine', 'valium']
freq_summary(all_drugs, 'any_legal')

core_drugs = ['cannabis', 'cocaine', 'ecstasy', 'mdma', 'ketamine', 'Mephedrone', 'speed']
freq_summary(core_drugs, 'core')

# Now poly drug use
illegal_drugs = ['cannabis', 'cocaine', 'ecstasy', 'mdma', 'ketamine', 'mushrooms', 'lsd', 'Mephedrone', '2cb', 'speed']
column_prefix = 'polydrug'
for period in PERIODS:
    labels = ['{}_{}'.format(drug, period) for drug in illegal_drugs]
    clabel = '{}_{}'.format(column_prefix, period)
    bt_df[clabel] = bt_df[labels].sum(axis=1) >= 2

# Now poly substance use - NEED TO DECIDE ON ALCOHOL COLUMN
# column_prefix = 'polysubstance'
# for period in PERIODS:
#     clabel = '{}_{}'.format(column_prefix, period)
#     labels = ['polydrug_{}'.format(period), 'polydrug_{}'.format(period)]
#     bt_df[clabel] = bt_df[labels].sum(axis=1) >= 2

# Delete the drug columns that don't match Liam's data
for drug in column_map_extra.values():
    labels = ["{}_{}".format(drug, period) for period in PERIODS]
    bt_df.drop(labels, axis=1, inplace=True)

# Covert all booleans to Yes/No strings
# 'polysubstance'
prefixes = list(column_map_core.values()) + ['any', 'any_legal', 'core', 'polydrug' ]
columns = []
for prefix in prefixes:
    for period in PERIODS:
        columns.append("{}_{}".format(prefix, period))
for column in columns:
    bt_df[column] = bt_df[column].map({True : 'yes', False : 'no'})
        
#bt_df.to_csv('foo.csv')
print("DONE DRUGS")

DONE DRUGS


In [8]:
# Now disentangle the disposals columns

# Map of Google Forms responses to Liam's categories
GMAP = {
    'I will ask the Loop to safely dispose of the rest of the sample in my possession' : 'a',
    'I will throw it away myself' : 'b',
    'I will take a smaller amount of it' : 'c',
    'I will take a larger amount of it' : 'd',
    'I will take the same amount as usual' : 'e',
    'I will take it over a longer time period' : 'f',
    'I will be more careful about mixing it with other substances' : 'g',
    'I will give it away instead of taking it myself' : 'h',
    'I will sell it' : 'i',
    'I will obtain more on site' : 'j',
    'I will warn my friends and acquaintances' : 'k',
    'I will warn others via social media and public websites' : 'l',
    'I will tell my dealer' : 'm',
    'I will return it to my dealer' : 'n',
    'I will ask for a refund from my dealer' : 'o',
    'I will go to another dealer' : 'p',
    'I will keep it to take it elsewhere, after the festival' : 'q',
    'I will do something else' : 'r'
}
CATEGORIES = list(sorted(GMAP.values()))

def get_disposal(cell):
    """Return a list of booleans depending on which categories were seen for this cell."""
    result = [False] * len(CATEGORIES)
    if isinstance(cell, float) and np.isnan(cell):
        return result # nan's is considered not having the value
    values = cell.split(',')
    for v in values:
        v = v.strip()
        if v in GMAP.keys():
            idx = CATEGORIES.index(GMAP[v])
            result[idx] = True
    return result

label1 = 'After hearing today’s test results and harm reduction advice from The Loop, what do you plan to do with the sample?'
label2 = 'What other actions will you do?'

# Create two Series with the results of parsing the two columns
series1 = bt_df[label1].apply(get_disposal).apply(pd.Series)
series2 = bt_df[label2].apply(get_disposal).apply(pd.Series)

# OR them to get the final column as a dataframe and name the columns accordingly
df_tmp = series1 | series2

# Rename columns to match
df_tmp.columns = CATEGORIES

# add to the bt_df
bt_df = pd.concat([bt_df, df_tmp], axis=1)

# CONVERT TRUE/FALSE TO Yes/No
for column in CATEGORIES:
    bt_df[column] = bt_df[column].map({True : 'yes', False : 'no'})

# Delete redundant columns
bt_df.drop([label1, label2], axis=1, inplace=True)

print("DONE DISPOSALS")

DONE DISPOSALS


In [9]:
# Fix remaining non-matching columns

# ConcernsWithCurrentFeelings' and any non yes/no answers to'WhatConcerns'
label1 = 'ConcernsWithCurrentFeelings'
label2 = 'WhatConcerns'
bt_df[label1].fillna('No', inplace=True)
mask = bt_df[label1] == 'No'
bt_df[label2] = np.nan
bt_df[label2].where(mask, bt_df[label1], inplace=True) # Copy values from label1 column over
bt_df[label1].where(mask, 'Yes', inplace=True) # Set copied over values to 'Yes'

label1 = 'PriorConcerns'
label2 = 'Why'
bt_df[label1].fillna('No', inplace=True)
mask = bt_df[label1] == 'No'
bt_df[label2] = np.nan
bt_df[label2].where(mask, bt_df[label1], inplace=True) # Copy values from label1 column over
bt_df[label1].where(mask, 'Yes', inplace=True) # Set copied over values to 'Yes'

# Fix Ethnicity / Ethnicity_Other
label1 = 'Ethnicity'
label2 = 'Ethnicity_other'
#Find where the values aren't the 5 core
mask = bt_df[label1].apply(lambda x: x not in ['White', 'Black', 'Asian', 'Mixed Race', 'Other'])
bt_df[label2] = np.nan
bt_df[label2].where(~mask, bt_df[label1], inplace=True) # Copy values from label1 column over
bt_df[label1].where(~mask, 'Other', inplace=True) # Set copied over values to 'Other'

# Can't currently process data for other_ or polysubstance_ so just set to nan
columns = []
for prefix in ['other', 'polysubstance']:
    for period in PERIODS:
        columns.append("{}_{}".format(prefix, period))
# Add null columns
bt_df = pd.concat([bt_df, pd.DataFrame(columns=columns)], axis='columns')

# For now we just delete the beer and cider and set BeerCider to na as we can't merge
bt_df.drop(['How much cider have you had today?', 'How much beer have you had today?'], axis=1, inplace=True)
bt_df['BeerCider'] = np.nan

# Can't calculate UnitsConsumed
bt_df['UnitsConsumed'] = np.nan

# PriorConcerns': and any non yes/no answers to 'Why'
print("DONE FIX COLUMNS")
#bt_df.to_csv('foo.csv')

DONE FIX COLUMNS


In [10]:
# Make sure columns match between two dataframes
assert(set(bt_df.columns.values) == set(spss_df.columns.values))

# Rename bt_df columns to match spss_df
bt_df = bt_df[spss_df.columns.values]

# Join the two dataframes
spss_df = pd.concat([spss_df, bt_df], axis='rows', ignore_index=True)

In [11]:
labdata = '/opt/random/Loop 2017 Lab fixed data.xlsm'
lab_df = pd.read_excel(labdata, sheet_name='Raw LabData')
print("Read in from LAB ",len(lab_df))

# Remame 'Event Name' and 'Sample Number' columns so they match
lab_df.rename(columns={'Event  Name': 'Festival', 'Sample Number': 'SampleNumber'}, inplace=True)

# Delete any rows where SampleNumber or Festival is NA as we can't do anything with it
lab_df.dropna(subset=['SampleNumber', 'Festival'], inplace=True) # This just drops one case

# Uppercase all sample numbers
labels = ['SampleNumber']
lab_df.loc[:, labels] = lab_df[labels].apply(lambda x: x.str.capitalize())

# Some sample numbers begin with W or F 
#print(len(lab_df[ ~ (lab_df['SampleNumber'].str.startswith('F') | lab_df['SampleNumber'].str.startswith('A')) ]))
print("Final LAB ",len(lab_df)) # shows we are left with 2543 datasets

Read in from LAB  2544
Final LAB  2543


In [12]:
dft = pd.merge(spss_df, lab_df, how='inner', on=['Festival','SampleNumber'])
print("%d entries were merged" % len(dft))

# For checking which entries can't be merged - check for right_only
#pd.merge(lab_df, spss_df, how='outer', indicator=True)

# Sort first by Festival, then SampleNumber
dft.sort_values(['Festival', 'SampleNumber'], ascending=True, inplace=True)

# Here we reorder columns that should be identical to:
# 1. spot data errors
# 2. remove duplicate columns once we're happy data is consistent
prefix_cols = ['Festival', 'SampleNumber',
             'Sample submission time', 'Date & Time of return', 'Date & Time of intervention', 
             'Client age', 'Age', 'Client gender', 'Gender', 'Bought as', 'SubmittedSubstanceAs']

# Get the list of columns excluding the ones in prefix_cols
cols = [c for c in dft.columns.tolist() if c not in prefix_cols]
# Prepend prefix_cols to create the new list
cols = prefix_cols + cols
# Reorder columns
dft = dft[cols]

# Uppercase all genders for consistency
labels = ['Client gender', 'Gender']
dft.loc[:, labels] = dft[labels].apply(lambda x: x.str.upper())
# Set any MISSING to be nan
dft.loc[:, labels] = dft.loc[:, labels].replace({'MISSING':np.nan})

# Dump to excel
writer = pd.ExcelWriter('merged.xlsx')
dft.to_excel(writer, 'MergedData', index=False)
writer.save()

1493 entries were merged


In [13]:
# # See which non-na ages don't match
# # 540 entries have valid ages
# print(len(dft))
# df = dft[pd.notnull(dft['Client age']) & pd.notnull(dft['Age'])]
# print(len(df))
# df = df[df['Client age'] != df['Age']]
# # 127 don't match
# print(len(df))
# # df.to_csv('foo.csv')

# Cross tab 'Client gender' and 'Gender
#print(dft['Client gender'].unique())
#print(dft['Gender'].unique())

# # Look where they don't match
# df = dft[pd.notnull(dft['Client gender']) & pd.notnull(dft['Gender'])]
# df = df[df['Client gender'] != df['Gender']]
# # 127 don't match
# print(len(df))
# df.to_csv('foo.csv')

In [74]:
# Check for duplicates in SPSS data (Lab data contains no duplicates)
for festival in ['BT2017', 'KC2017', 'SGP2017']:
    sample_numbers = spss_df.loc[spss_df['Festival'] == festival, ['SampleNumber']]
    indexes = sample_numbers[sample_numbers.duplicated(keep=False)].index
    print("Festival {}: {} entries {} duplicates".format(festival, len(sample_numbers), len(indexes)))
    df = spss_df.iloc[indexes,].sort_values('SampleNumber', ascending=True)
    df.to_csv('{}_duplicates.csv'.format(festival))

# BT2017:
# F0327 - looks like a genuine duplicate with small differences (e.g. UnitsConsumed, a,b,c etc.) - pick idx 767
# F0415 - duplicate - pick idx 1436 but need to manually curate
# F0526 - 2 samples? pick idx 1414
# F0877 - duplicate - use idx 1292
# F1205 - duplicate use id 1136
# F1267 - duplicate but need to manually curate
# F1271 - duplicate but 2 different drugs?
# F0004, F0015, F0029, F0120, F0256,  F0295, F0323, F0480 - different data

# KC2017
# F0109 - duplicate - 2nd has almost no data so use idx 6
# F0124 - different data

# SGP2017
# F0090 - duplicate but different drug - use idx 475
# F0148 - duplicate but different drug - use idx 376
# F0253 - duplicate - use idx 4
# F0510 - duplicate - use idx 563
# F0526 - duplicate - use idx 589
# 
# F0047, F0050, F0057, F0063, F0130, F0147, F0181, F0255, F0275, F0445, F0446, F0526, F0549, F0624

Festival BT2017: 873 entries 30 duplicates
Festival KC2017: 67 entries 4 duplicates
Festival SGP2017: 557 entries 36 duplicates


In [93]:
# Try matching duplicated sample numbers to lab results.
# Check for duplicates
#for festival in ['BT2017', 'KC2017', 'SGP2017']:
pd.set_option('expand_frame_repr', False)
for festival in ['BT2017']:
    sample_numbers = spss_df.loc[spss_df['Festival'] == festival, ['SampleNumber']]
    duplicated = sample_numbers[sample_numbers.duplicated(keep='first')]['SampleNumber']
    print("Festival {}: {} entries {} duplicates".format(festival, len(sample_numbers), len(indexes)))
    for sn in duplicated:
        print("CHECKING ",sn)
        print(spss_df.loc[(spss_df['Festival'] == festival) & (spss_df['SampleNumber'] == sn),
                          ['Age', 'Gender', 'Date & Time of intervention', 'SubmittedSubstanceAs']])
        print(lab_df.loc[(lab_df['Festival'] == festival) & (lab_df['SampleNumber'] == sn),
                        ['Client age', 'Client gender', 'Date & Time of return', 'Bought as', 'Client suspicion', 'Matches Sold as', 'Final Result']])




Festival BT2017: 873 entries 2 duplicates
CHECKING  F0327
      Age  Gender Date & Time of intervention SubmittedSubstanceAs
767  19.0  Female 2017-08-11                   Ketamine           
768  19.0  Female 2017-08-11                   Ketamine           
      Client age Client gender Date & Time of return Bought as Client suspicion  Matches Sold as Final Result
1734 NaN          NaN           2017-08-11 17:03:44   Ketamine  Methoxetamine   NaN               Ketamine   
CHECKING  F0120
      Age  Gender Date & Time of intervention SubmittedSubstanceAs
653  24.0  Female 2017-08-11 14:50:00          MDMA               
844  18.0  Male   2017-08-11 19:47:00          MDMA               
      Client age Client gender Date & Time of return Bought as Client suspicion  Matches Sold as Final Result
1527 NaN          NaN           2017-08-11 14:34:44   MDMA      NaN             NaN               MDMA       
CHECKING  F1267
      Age  Gender Date & Time of intervention SubmittedSubstanceAs
9

/Users/jmht/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  
/Users/jmht/miniconda2/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)



      Client age Client gender Date & Time of return Bought as Client suspicion  Matches Sold as Final Result
2405 NaN          NaN           2017-08-12 20:20:40   MDMA      NaN             NaN               MDMA       
CHECKING  F0480
       Age Gender Date & Time of intervention SubmittedSubstanceAs
980   20.0  Male  2017-08-12 16:15:00          MDMA               
1240  22.0  Male  2017-08-12 16:30:00          MDMA               
      Client age Client gender Date & Time of return Bought as Client suspicion  Matches Sold as Final Result
1887 NaN          NaN           2017-08-12 16:14:14   MDMA      NaN             NaN               Cocaine    
CHECKING  F0877
       Age  Gender Date & Time of intervention SubmittedSubstanceAs
1289  22.0  Female 2017-08-12 17:15:00          MDMA               
1292  22.0  Female 2017-08-12 17:15:00          MDMA               
      Client age Client gender Date & Time of return Bought as Client suspicion  Matches Sold as Final Result
2279 NaN    

In [31]:
# s = spss_df.columns.values
# b = bt_df.columns.values
# print(set(s).difference(set(b)))
# print(set(b).difference(set(s)))

#spss_df['other_specify'].unique()
#print(bt_df['Have you ever taken any other drugs I didn\'t mention?'].unique())

# Need to check drug names match across all datasets
def unique(column):
    return sorted(set([a.strip().lower() for a in column.astype(str).unique()]))

bought_as = unique(lab_df['Bought as'])
client_suspicion = unique(lab_df['Client suspicion'])
final_result = unique(lab_df['Final Result'])
submitted_as = unique(spss_df['SubmittedSubstanceAs'])
other_specify = unique(spss_df['other_specify'])

# print("Bought as:", bought_as)
# print("Client suspicion:", client_suspicion)
# print("Final Result:", final_result)
# print("SubmittedSubstanceAs:", submitted_as)
# print("other_specify:", other_specify)

x = bought_as + client_suspicion + final_result + submitted_as + other_specify
x = sorted(set(x))
print("X: ",x)

AMPHETAMINE = 'amphetamine'
ANALYSIS_ICON = 'analysis_inconclusive'
COCAINE = 'cocaine'
FOUND = 'found'
KETAMINE = 'ketamine'
LSD = 'lsd'
MEPHEDRONE = 'mephedrone'
MDMA = 'mdma'
NETHYLPENTYLONE = 'n-ethylpentylone'
UNKNOWN = 'unknown'

drugs_map = { 
    AMPHETAMINE : ['speed', 'Speed', 'base/speed'],
    ANALYSIS_ICON : ['compound not in library', 'inconclusive', 'insufficient quantity for testing', 
                     'insufficient sample', 'insufficient sample', 'no active component identified', 
                     'no match', 'no match', 'none', 'result missing', 'unable to test', 'unknown'],
    COCAINE : ['cocaine', 'coke', 'cut cocaine'],
    FOUND : ['found', 'unknow found'],
    KETAMINE : ['ketamine', '?ket', '/ketamie', 'maybe ketamine?', 'katamine', 'vanila ketamine', 
                'vetamine', 'not mdma. ketamine?', 'ketamoine'],
    LSD : ['acid', 'liquid lsd', 'lsd'],
    MEPHEDRONE : ['meow meow', 'mcat', 'mephedrone'],
    MDMA : ['mdma', 'mdxx', 'mda/mdea/mdma', 'mdma,', 'mandy', 'probaby mdma', 'mdma?'],
    NETHYLPENTYLONE : ['n-ethylpentylone'],
    UNKNOWN : ['unknown', 'unknown pill', 'unsure', 'unsure - maybe dmt', 'unsure of content', 
               'no effect', 'no idea'],   
    }


X:  ['', '/ketamie', '2 cb', '2-cb or pma', '25-1, in last year; salvia, more than a year; 2c-x, more than a year; research chemical, in the last year.', '25i', '2bx_salvia_benzo', '2c-b', '2c-b-fly', '2c-i', '2c-t-7', '2cb', '2cb_2ce_2ci', '2cb_2ci_dmt', '2cb_2ci_valium_cathinanes_rcs', '2cb_2si', '2cb_5homet_valium', '2cb_amphetamine', '2cb_amphetamine_dmt', '2cb_amphetamine_valium_truffle_tramadol', '2cb_diazepam', '2cb_diazepam_speed', '2cb_diazepam_speed_opiods', '2cb_dmt_salvia', '2cb_meth', '2cb_opiates_speed_diazepam', '2cb_opium_valium', '2cb_researchchems', '2cb_speed_benzo_diazepam', '2cb_speed_opiates', '2cb_speed_opioids_valium', '2cb_speed_valium_opiates', '2cb_valium_ahcodmt', '2cb_valium_opioid_speed', '2cb_valium_speed_meth', '2cb_valium_speed_opiods', '2cb_vallium_speed', '2cb_xanax', '2cb_xanax_amphetamine_dmt', '2ci', '2ci, 5-ho met, dmt,', '2ci_dmt_2si_speed_opium', '2cx', '3/4 of pill green shooting star', '4 aco dmt  dmt phenibut', '4-aco dmt', '4-aco-dmt', '4-ch